Previous experiment with training LoRAs weren't possible on the Mac, so just going to run a regular stable diffusion model.

Based on advice from ChatGPT and my own research:

- Next.js as it is a full package and is optimal for SEO
- FastAPI (Python) for backend IF NEEDED due to familiarity and it's a modern technology
- TailwindCSS for styling
- EC2 for running the AI model (Stable Diffusion)
- SQS for queueing system for now, will implement Kafka later
- Redis useful for CV and good at general jobs, will implement later
- DynamoDB for general storage
- Firebase Auth + Stripe for authorisation and billing
- CloudFront for CDN
- ComfyUI + LoRA for generating images

Next.js setup:

npx create-next-app@latest my-ai-app
cd my-ai-app

say yes to all options

cd my-ai-app to run